In [1]:
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
#matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({"font.size":16})
import os
import sys

sys.path.append(os.path.abspath('..'))
from modules import io
import modules.vascular_data as sv
import modules.sv_image as sv_image

In [2]:
W       = 160
SPACING = 0.029
WI = W*SPACING/2
images = {
    "144":"/home/marsdenlab/projects/SV/user_variation/0144/0144_gt/Images/0144.vti",
    "171":"/home/marsdenlab/projects/SV/user_variation/171/0171_1/Images/0171.vti",
    "cabg6":"/home/marsdenlab/projects/SV/user_variation/cabg6/cabg6_1/Images/cabg6.vti",
    "kdr33":"/home/marsdenlab/projects/SV/user_variation/kdr_nn/KDR33/Images/kdr33.vti",
    "SU0259_2011":"/home/marsdenlab/projects/SV/user_variation/pulm_melody/SU0259_2011/Images/SU0259_2011_45184086_cm.vti"
}

sv_images = {}
for k,v in images.items(): 
    sv_images[k] = sv_image.Image(v)
    sv_images[k].set_reslice_ext(W)
    sv_images[k].set_spacing(SPACING)


user_csv  = "/home/marsdenlab/projects/SV/user_variation/results_drop/user.csv"
DF        = pd.read_csv(user_csv)

json_dir = "/home/marsdenlab/projects/SV/user_variation/json_after"
json_files = os.listdir(json_dir)
json_files = [json_dir + '/'+f for f in json_files]
print(json_files[:10])

['/home/marsdenlab/projects/SV/user_variation/json_after/kdr33.8.9.mr.aorta.10.json', '/home/marsdenlab/projects/SV/user_variation/json_after/cabg6.3.1.ct.rc1.66.json', '/home/marsdenlab/projects/SV/user_variation/json_after/SU0259_2011.11.12.mr.RPA_02.19.json', '/home/marsdenlab/projects/SV/user_variation/json_after/cabg6.2.3.ct.rc1.6.json', '/home/marsdenlab/projects/SV/user_variation/json_after/cabg6.1.2.ct.lc1.105.json', '/home/marsdenlab/projects/SV/user_variation/json_after/171.2.3.mr.l_cerebral.105.json', '/home/marsdenlab/projects/SV/user_variation/json_after/kdr33.8.9.mr.brach1.100.json', '/home/marsdenlab/projects/SV/user_variation/json_after/171.3.nn.mr.l_cerebral.70.json', '/home/marsdenlab/projects/SV/user_variation/json_after/144.6.nn.ct.aorta.369.json', '/home/marsdenlab/projects/SV/user_variation/json_after/kdr33.9.nn.mr.rca.60.json']


In [4]:
seen = {}
C = []
X = []
data = []
for i in range(DF.shape[0]):
    row   = DF.iloc[i]
    image = row['image']
    path  = row['path']
    point = row['point']
    code  = path+'.'+str(point)   
    
    imcode = image+'.'+path+'.'+str(point)
    if imcode in seen:
        continue
    seen[imcode]=1
    
    jfiles = [f for f in json_files if ('.'+code+'.') in f and image in f]
    jsons = [io.load_json(f) for f in jfiles]
    
    user_jsons = [j for j in jsons if not 'nn' in j['code']]
    
    p = user_jsons[0]['p']
    n = user_jsons[0]['tangent']
    t = user_jsons[0]['rotation']

    img = sv_images[image].get_reslice(p,n,t)
    
    X.append(img)
    
    contours = []
    for j in user_jsons:
        c = np.array(j['contour2D'])
        contours.append(c)
    C.append(contours)

    data.append({
        "image":image,
        "path":path,
        "point":point,
        "code":code
    })
    
C = np.array(C)
X = np.array(X)

In [5]:
print(C.shape)
print(X.shape)

np.save('./UQ/user_data/X.npy',X)
np.save('./UQ/user_data/C.npy',C)

(308, 3)
(308, 160, 160)


In [7]:
df = pd.DataFrame(data)
df.to_csv('./UQ/user_data/data.csv')